In [1]:
import os
import json
from dotenv import load_dotenv
load_dotenv()
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")
redirect_uri = 'http://localhost:3000/spotify_return'
device_name = 'Pi-Music-Server Spotify Player'

In [2]:
import requests

In [91]:
scopes = [
    'app-remote-control',
    'streaming',
    'user-read-private',
    'user-read-email',
    'user-read-playback-state',
    'user-modify-playback-state',
    'user-read-currently-playing',
    'user-library-read',
]
print("%20".join(scopes))

app-remote-control%20streaming%20user-read-private%20user-read-email%20user-read-playback-state%20user-modify-playback-state%20user-read-currently-playing%20user-library-read


In [92]:
url_for_user = f'https://accounts.spotify.com/authorize?client_id={SPOTIFY_CLIENT_ID}&response_type=code&redirect_uri={redirect_uri}&scope={"%20".join(scopes)}&state=abcd'
print(url_for_user)

https://accounts.spotify.com/authorize?client_id=cd9f7d934468415b83c0e9c6d18c60cf&response_type=code&redirect_uri=http://localhost:3000/spotify_return&scope=app-remote-control%20streaming%20user-read-private%20user-read-email%20user-read-playback-state%20user-modify-playback-state%20user-read-currently-playing%20user-library-read&state=abcd


In [7]:
auth_code = 'AQA1SJ0D2_rBDHzsTF0Can3IaO-Sl0WWFGDcFGGMh-kTXTsAv4vTuLIPkRPNN6VQnUM8q_90UXS0TxsEBGaf_PKrAHUaD4v3zc-avk1L5Lgl_hN5YLJzzpDrX59gyNdvYXNLH70wR9g2W185NprzmD8be-L7SLew8tR2jcZpQ27s59qoylztA-kGfWGwacDp3i-HOwn4E1ZBjD7NqDpN56GLR-CT14OoKSvRWV1PIs-gs9mMXMd7s47ypYSBD63f3G3julLyt0kM972P-Xq27rPZ1dw1UdhcibXG6r8'
token_url='https://accounts.spotify.com/api/token'
print(token_url)

x = requests.post(token_url, data = {
    'grant_type': 'authorization_code',
    'code': auth_code,
    'redirect_uri': redirect_uri,
    'client_id': SPOTIFY_CLIENT_ID,
    'client_secret': SPOTIFY_CLIENT_SECRET,
})

https://accounts.spotify.com/api/token
<class 'requests.models.Response'>


In [6]:
result = json.loads(x.text)
access_token = result['access_token']
print(access_token)
print(x.text)

BQAP-bT1p4ruMRMWo-kkAcecOf-7d3OyNh-7j-XGhVqK3QO7oTJaMPTwNjTjZvrnjYkqSTj7RibU24IIQqJMEOEJNgPDOz_vCfygxoVIwCyYEn6K773TDClM0c51_BSsBknTQ9nF23KXY1WkIawtdpyy7olWVy2a9cv5hqwz4g
{"access_token":"BQAP-bT1p4ruMRMWo-kkAcecOf-7d3OyNh-7j-XGhVqK3QO7oTJaMPTwNjTjZvrnjYkqSTj7RibU24IIQqJMEOEJNgPDOz_vCfygxoVIwCyYEn6K773TDClM0c51_BSsBknTQ9nF23KXY1WkIawtdpyy7olWVy2a9cv5hqwz4g","token_type":"Bearer","expires_in":3600,"refresh_token":"AQANh2RsNnaTxcEtK4xGFvWR2dK7oUF6uw4QihhitGGSjRpZNQaWzk4o-JrlZhfA4Tir4yFZXiAjLlfZicRGM73T2VtxatPyh_M-ZsB_q2aYgQeqZzKZJyXRDxEjwEPUucc","scope":"app-remote-control streaming user-read-email user-read-private"}


In [10]:
to_add_str = '''
<!DOCTYPE html>
<html>
<head>
  <title>Spotify Web Playback SDK Quick Start Tutorial</title>
</head>
<body>
  <h1>Spotify Web Playback SDK Quick Start Tutorial</h1>
  <h2>Open your console log: <code>View > Developer > JavaScript Console</code></h2>

  <script src="https://sdk.scdn.co/spotify-player.js"></script>
  <script>
    let player;
    window.onSpotifyWebPlaybackSDKReady = () => {
      const token = \'''' + access_token + "'" + ''';
      player = new Spotify.Player({
        name: \'''' + device_name + "'" + ''',
        getOAuthToken: cb => { cb(token); }
      });

      // Error handling
      player.addListener('initialization_error', ({ message }) => { console.error(message); });
      player.addListener('authentication_error', ({ message }) => { console.error(message); });
      player.addListener('account_error', ({ message }) => { console.error(message); });
      player.addListener('playback_error', ({ message }) => { console.error(message); });

      // Playback status updates
      player.addListener('player_state_changed', state => { console.log(state); });

      // Ready
      player.addListener('ready', ({ device_id }) => {
        console.log('Ready with Device ID', device_id);
      });

      // Not Ready
      player.addListener('not_ready', ({ device_id }) => {
        console.log('Device ID has gone offline', device_id);
      });

      // Connect to the player!
      player.connect();
    };
  </script>
</body>
</html>
'''

In [69]:
out_file = open('spot_play.html', 'w')
out_file.write(to_add_str)
out_file.close()

In [15]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

In [13]:
options = Options()
options.headless = False
browser = webdriver.Firefox(options=options)
#browser.get(f'data:text/html;charset=utf-8,{to_add_str}')
browser.get('soundcloud.com')

InvalidArgumentException: Message: Malformed URL: URL constructor: soundcloud.com is not a valid URL.


In [148]:
# viewing device list: 
# https://developer.spotify.com/documentation/web-api/guides/using-connect-web-api/
base_url = 'https://api.spotify.com/v1/me/player/devices'
token = 'BQD9MH_qMsG_2E-OAFMbJwXI8BNlDbCkShixavlnoO0LCzVRtU_o1d3adeEjnWIxDqfvGALQEElbAhGS6E5Dw4xFFFDsA7n_BdCWTd3IsasbUH-EguOj3luJX4sYGDsvmpE-Ardpejz-QT8iELVC8umeMPOa3PaDG4V7LRxmOG8mNengWRHg0oA'
x = json.loads(requests.get(base_url, headers = {'Authorization': f'Bearer {token}'}).content)

pi_music_server_device = None
for device in x['devices']:
    if device['name'] == 'Pi-Music-Server Spotify Player':
        pi_music_server_device = device
print(pi_music_server_device)

None


In [144]:
base_url = 'https://api.spotify.com/v1/me/player'
data_dic = '{device_ids: ["882d07103c0b9fea360ff3caaa465828261f82a3"]}'
print(data_dic)
x = requests.get(base_url, data = data_dic,  headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'})
print(x)
print(x.content)

{"device_ids": ["882d07103c0b9fea360ff3caaa465828261f82a3"]}
<Response [400]>
b'<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 400 (Bad Request)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/

In [131]:
print(json.loads(x.content))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [115]:
# transferring control over to device_name:
# https://developer.spotify.com/documentation/web-api/reference/player/transfer-a-users-playback/
base_url = 'https://api.spotify.com/v1/me/player'
x = requests.get(base_url, data = {
    'device_ids': f'[{pi_music_server_device["id"]}]'
}, headers = {
    'Authorization': f'Bearer {token}',
    
})

In [116]:
print(x)
print(x.content)

<Response [400]>
b'<!DOCTYPE html>\n<html lang=en>\n  <meta charset=utf-8>\n  <meta name=viewport content="initial-scale=1, minimum-scale=1, width=device-width">\n  <title>Error 400 (Bad Request)!!1</title>\n  <style>\n    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-

In [ ]:
# open file in selenium


In [ ]:
# create following functions using browser scripts (player object made visible):
# https://developer.spotify.com/documentation/web-playback-sdk/reference/#api-spotify-player-toggleplay

#def togglePlay():

#def playSong(song_id):

In [42]:
import spotipy

In [151]:
sp = spotipy.Spotify(auth=token)
print(sp.devices())
#sp.transfer_playback('882d07103c0b9fea360ff3caaa465828261f82a3')

{'devices': [{'id': '882d07103c0b9fea360ff3caaa465828261f82a3', 'is_active': False, 'is_private_session': False, 'is_restricted': False, 'name': 'Pi-Music-Server Spotify Player', 'type': 'Computer', 'volume_percent': 100}]}


HTTP Error for GET to https://api.spotify.com/v1/me/player/devices returned 401 due to Permissions missing


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/me/player/devices:
 Permissions missing